## Installing Required Packages

In [ ]:
!pip install SpeechRecognition wavio ffmpeg-python gtts
!mkdir sounds
!wget https://raw.githubusercontent.com/myprogrammerpersonality/Voice_Recognition/master/Template.csv

     |████████████████████████████████| 32.8MB 120kB/s 
  Created wheel for gtts-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4097 sha256=5b64976846ddd88a6f0f72af6eb2f8542fc9d047d1ec76c81c24e33b4b2d48ce
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
Successfully built gtts-token
--2020-08-07 13:28:56--  https://raw.githubusercontent.com/myprogrammerpersonality/Voice_Recognition/master/Template.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267 [text/plain]
Saving to: ‘Template.csv’

Template.csv        100%[===================>]     267  --.-KB/s    in 0s      

2020-08-07 13:28:56 (12.8 MB/s) - ‘Template.csv’ saved [267/267]



## Import Packages and Define Functions

In [ ]:
import scipy
from scipy.io.wavfile import read as wav_read
import io
from IPython.display import HTML, Audio, clear_output
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import ffmpeg
import IPython.display as ipd
from IPython.display import Javascript
import speech_recognition as sr
import matplotlib.pyplot as plt
import time
import wavio
import pandas as pd
from gtts import gTTS #Import Google Text to Speech

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""
output_html = """<style>
    fieldset {{
    font-family: sans-serif;
    border: 5px solid #1F497D;
    background: #ddd;
    border-radius: 5px;
    padding: 15px;
}}

fieldset legend {{
    background: #1F497D;
    color: #fff;
    padding: 5px 10px ;
    font-size: 32px;
    border-radius: 10px;
    box-shadow: 0 0 0 5px #ddd;
    margin-left: 20px;
}}
</style>

<section style="margin: 15px;">
<fieldset style="min-height:100px;">
<legend><b> {} </b> </legend>
<label> <h1 style="font-size: 80px;float: top;">{} ==> Sample {}</h1><br/> </label>
</fieldset>"""

def record(sec=3, file_name = 'temp.wav', verbose=False):
    if verbose: print('Start Recording :')
    display(Javascript(RECORD))
    s = eval_js('record(%d)' % (sec*1000))
    b = b64decode(s.split(',')[1])

    process = (ffmpeg
        .input('pipe:0')
        .output('pipe:1', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True))
    output, err = process.communicate(input=b)
  
    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    if verbose: print('Recording Finished')
    return audio, sr

def hearing(step_sec = 5, key_word = 'go', stop_word = 'stop', verbose = False):
    key = key_word.lower()
    key_stop = stop_word.lower()
    num = 0
    while True:
        num += 1
        if verbose: print(f'Round{num}')

        # Part 1: Recording
        t1 = time.time()
        audio, sound_rate = record(sec=step_sec, verbose=False)
        
        # Part 2: Saving Audio File
        t2 = time.time()
        wavio.write('sound.wav', audio, sound_rate)

        # Part 3: Try to Recognize and Check for Key_Word
        t3 = time.time()
        r = sr.Recognizer()
        with sr.WavFile('sound.wav') as source:
            audio = r.record(source)
        try:
            text = r.recognize_google(audio)
            text = text.lower()
            if verbose >= 2: print(f'You Said :{text}')
            if key in text:
                return 1
            if key_stop in text:
                return 0
        except:
            pass
        
        if verbose:print(f'Part 1 {t2-t1}')
        if verbose:print(f'Part 2 {t3-t2}')
        if verbose:print(f'Part 3 {time.time()-t3}')

## Text to Speech

In [ ]:
data = pd.read_csv('Template.csv')

main_dict = {}
for name in data['Metabolite']:
    vols = list(data[data['Metabolite']==name].iloc[:,1:].values[0])
    main_dict[name] = [vols, sorted(range(len(vols)), key=lambda k: vols[k])]

for name in main_dict.keys():
    tts = gTTS('Start Aliquoting {}'.format(name)) #Provide the string to convert to speech
    tts.save('sounds/{}.wav'.format(name)) #save the string converted to speech as a .wav file
    for i, vol in enumerate(main_dict[name][0]):
        tts = gTTS('{} in Sample {}'.format(vol, i+1))
        tts.save('sounds/{}_{}.wav'.format(name, i))

## Main Part

In [7]:
# sorted version within each metabolite
for name in main_dict.keys():
    print('Start Aliquoting ', name)
    display(Audio('sounds/{}.wav'.format(name), autoplay=True))
    display(HTML(output_html.format(name, "#", "#")))
    time.sleep(4)
    clear_output(wait=True)
    time.sleep(2)
    for i in range(len(main_dict[name][0])):
        display(Audio('sounds/{}_{}.wav'.format(name, main_dict[name][1][i]), autoplay=True))
        display(HTML(output_html.format(name, main_dict[name][0][main_dict[name][1][i]], main_dict[name][1][i]+1)))
        if hearing(step_sec=5, key_word='go', stop_word='stop', verbose=2):
            pass
        else:
            clear_output(wait=True)
            break
        clear_output(wait=True)

Round1


<IPython.core.display.Javascript object>

You Said :stop
